In [1]:
from datasets import load_dataset
import torchaudio

# Load your dataset
# Replace with the actual path to your dataset or use Hugging Face's datasets library
dataset = load_dataset("https://onedrive.live.com/?id=BA63D4A440C2FA9%21s1b49da28d90540b8a44655eb5d8b77fb&cid=0BA63D4A440C2FA9&redeem=aHR0cHM6Ly8xZHJ2Lm1zL2YvYy8wYmE2M2Q0YTQ0MGMyZmE5L0VpamFTUnNGMmJoQXBFWlY2MTJMZF9zQnZXV2c3RF9mMDE0ZWZSSDZFVFhjLUE%5FZT13QnNMVzk")

# Preprocess audio (resample, normalize, etc.)
def preprocess_audio(batch):
    audio = torchaudio.load(batch["audio_filepath"])[0]  # Adjust based on dataset structure
    batch["input_values"] = processor(audio, sampling_rate=16000).input_values[0]
    return batch

dataset = dataset.map(preprocess_audio)


c:\Users\mitta\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: Couldn't find any data file at c:\Users\mitta\OneDrive - iiit-b\Documents\ML-Fiesta\model\https:\onedrive.live.com\?id=BA63D4A440C2FA9%21s1b49da28d90540b8a44655eb5d8b77fb&cid=0BA63D4A440C2FA9&redeem=aHR0cHM6Ly8xZHJ2Lm1zL2YvYy8wYmE2M2Q0YTQ0MGMyZmE5L0VpamFTUnNGMmJoQXBFWlY2MTJMZF9zQnZXV2c3RF9mMDE0ZWZSSDZFVFhjLUE%5FZT13QnNMVzk.

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Trainer, TrainingArguments

# Load pre-trained ASR model and processor
model_name = "facebook/wav2vec2-large-xlsr-53"  # Example, use a model trained in regional languages if possible
model = Wav2Vec2ForCTC.from_pretrained(model_name)
processor = Wav2Vec2Processor.from_pretrained(model_name)

# Training arguments
training_args = TrainingArguments(
    output_dir="./asr_model",
    per_device_train_batch_size=16,
    evaluation_strategy="steps",
    num_train_epochs=3,
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    learning_rate=3e-4,
    warmup_steps=500,
)

# Trainer setup
def compute_metrics(pred):
    pred_ids = pred.predictions.argmax(-1)
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

trainer = Trainer(
    model=model,
    data_collator=processor,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()


In [ ]:
model.save_pretrained("asr_kannada_model")
processor.save_pretrained("asr_kannada_model")


In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch

# Load ASR model and processor
model = Wav2Vec2ForCTC.from_pretrained("asr_kannada_model")
processor = Wav2Vec2Processor.from_pretrained("asr_kannada_model")

# Function to transcribe audio file
def transcribe_audio(audio_path):
    audio, _ = torchaudio.load(audio_path)
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(inputs.input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    return processor.decode(predicted_ids[0])

# Transcribe all audio files in the corpus
corpus_transcriptions = []
for audio_path in corpus_audio_paths:  # Replace with your audio paths
    transcript = transcribe_audio(audio_path)
    corpus_transcriptions.append((audio_path, transcript))


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load pre-trained BERT model for embeddings
embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Embed corpus transcripts
corpus_embeddings = embedder.encode([text for _, text in corpus_transcriptions], convert_to_tensor=True)

# Function to answer a question
def answer_question(question_audio):
    # Transcribe question
    question_text = transcribe_audio(question_audio)
    
    # Compute embedding for question
    question_embedding = embedder.encode(question_text, convert_to_tensor=True)
    
    # Find closest matching audio in the corpus
    similarities = util.pytorch_cos_sim(question_embedding, corpus_embeddings)
    closest_idx = similarities.argmax()
    
    # Return the corresponding audio file and text
    closest_audio_path, closest_text = corpus_transcriptions[closest_idx]
    return closest_audio_path, closest_text

# Example usage
answer_audio_path, answer_text = answer_question("path/to/user_question_audio.wav")
print(f"Answer found in audio: {answer_audio_path}")
print("Answer text:", answer_text)
